In [2]:
# Step 1: Install dependency
!pip install ffmpeg-python

# Step 2: Clone the Wav2Lip repository
!git clone https://github.com/justinjohn0306/Wav2Lip

# Step 3: Download pretrained model (Wav2Lip_gan.pdh)
import requests
url = "https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA"
response = requests.get(url)

with open("Wav2Lip/checkpoints/wav2lip_gan.pth", "wb") as f:
    f.write(response.content)
    
# Step 4: Install the required dependencies for Wav2Lip
!cd Wav2Lip && pip install -r requirements.txt
!pip install pyaudio


# Step 5: Download pretrained model for face detection
url = "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth"
response = requests.get(url)

with open("Wav2Lip/face_detection/detection/sfd/s3fd.pth", "wb") as f:
    f.write(response.content)

Cloning into 'Wav2Lip'...


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu118
  Using cached https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl (13 kB)
  Cloning https://github.com/elliottzheng/batch-face.git (to revision master) to c:\users\lenovo\appdata\local\temp\pip-req-build-jbunafmt
  Resolved https://github.com/elliottzheng/batch-face.git to commit 376a9d4355686d904722ed2770502f92c861109a
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached opencv_contrib_python-4.10.0.82-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached opencv_python-4.10.0.82-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached https://download.pytorch.org/whl/cu118/torch-2.3.0%2Bcu118-cp311-cp311-win_amd64.whl (2673.0 MB)


  Running command git clone --filter=blob:none --quiet https://github.com/elliottzheng/batch-face.git 'C:\Users\Lenovo\AppData\Local\Temp\pip-req-build-jbunafmt'


   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   ------- ------------------------------- 30.7/164.1 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 164.1/164.1 kB 2.5 MB/s eta 0:00:00


### Before this step you need to install FFmpeg from official site
### After add system env variable "Path" with path to bin folder in FFmpeg 
### Example: Path - C:\Users\User\Documents\Projects\ffmpeg-master-latest-win64-gpl-shared\bin

In [1]:
import os
import subprocess
from urllib import parse as urlparse

# Step 1: Install yt-dlp
subprocess.run(['pip', 'install', 'yt-dlp'])

# Step 2: Define YouTube URL and Video ID
YOUTUBE_URL = 'https://www.youtube.com/watch?v=NyrZgEChI3k'
url_data = urlparse.urlparse(YOUTUBE_URL)
query = urlparse.parse_qs(url_data.query)
YOUTUBE_ID = query["v"][0]

# Remove previous input video
if os.path.isfile('input_vid.mp4'):
    os.remove('input_vid.mp4')

# Trim video (start, end) seconds
start = 0
end = 6
interval = end - start

# Step 3: Download and trim the YouTube video
subprocess.run(['yt-dlp', '-f', 'bestvideo[ext=mp4]', '--output', "youtube.%(ext)s", f'https://www.youtube.com/watch?v={YOUTUBE_ID}'])

# Cut the video using FFmpeg
subprocess.run(['ffmpeg', '-y', '-i', 'youtube.mp4', '-ss', str(0), '-t', str(6), '-async', '1', 'input_vid.mp4'])

# Display video.
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""<video width=600 controls><source src="{data_url}"></video>""")

# Preview the trimmed video
show_video('input_vid.mp4')

In [3]:
import os
from IPython.display import Audio
from IPython.display import display

upload_method = 'Path'  # Change this to 'Record' or 'Path'

# Remove previous input audio
if os.path.isfile('input_audio.wav'):
    os.remove('input_audio.wav')

def display_audio():
    display(Audio('input_audio.wav'))

if upload_method == 'Record':
    import pyaudio
    import wave

    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "input_audio.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    display_audio()

elif upload_method == 'Path':
    # Add the full path to your audio
    PATH_TO_YOUR_AUDIO = 'C:\\Users\\Lenovo\\Documents\\Bootcamp\\Projects\\Final_Project\\source_files\\putin.wav'

    # Load audio with specified sampling rate
    import librosa
    audio, sr = librosa.load(PATH_TO_YOUR_AUDIO, sr=None)

    # Save audio with specified sampling rate
    import soundfile as sf
    sf.write('input_audio.wav', audio, sr, format='wav')

    display_audio()

In [19]:
# os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [11]:
# Define the parameters for the Wav2Lip model
pad_top = 0
pad_bottom = 10
pad_left = 0
pad_right = 0
rescaleFactor = 1
nosmooth = False

# Set the full path to the Wav2Lip model and input files
checkpoint_path = "C:\\Users\\Lenovo\\Documents\\Bootcamp\\Projects\\Final_Project\\Wav2Lip\\checkpoints\\wav2lip_gan.pth"
input_face = "C:\\Users\\Lenovo\\Documents\\Bootcamp\\Projects\\Final_Project\\input_vid.mp4"
input_audio = "C:\\Users\\Lenovo\\Documents\\Bootcamp\\Projects\\Final_Project\\input_audio.wav"
results_path = "C:\\Users\\Lenovo\\Documents\\Bootcamp\\Projects\\Final_Project\\Wav2Lip\\results"

# Run the Wav2Lip model
# {"--nosmooth" if nosmooth else ""}
!cd Wav2Lip && python inference.py --checkpoint_path {checkpoint_path} --face {input_face} --audio {input_audio} --pads {pad_top} {pad_bottom} {pad_left} {pad_right} --resize_factor {rescaleFactor}

# Preview the output video
print("Final Video Preview")
print("Find the output video at", 'Wav2Lip/results/result_voice.mp4')
show_video('C:\\Users\\Lenovo\\Documents\\Bootcamp\\Projects\\Final_Project\\Wav2Lip\\results\\result_voice.mp4')

Using cuda for inference.
Load checkpoint from: C:\Users\Lenovo\Documents\Bootcamp\Projects\Final_Project\Wav2Lip\checkpoints\wav2lip_gan.pth
Models loaded
Reading video frames...
Number of frames available for inference: 150
(80, 513)
Length of mel chunks: 157
face detect time: 58.56116199493408
wav2lip prediction time: 69.21900916099548
Final Video Preview
Find the output video at Wav2Lip/results/result_voice.mp4



100%|██████████| 2/2 [01:09<00:00, 34.61s/it]
ffmpeg version N-115551-g4037d5e103-20240603 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (crosstool-NG 1.26.0.65_ecc5e41)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --enable-shared --disable-static --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --ena